<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/226_Cross_Sell_Upsell_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scoring utilities for Tier 2 Cross-Sell & Upsell Orchestrator

In [ ]:
"""Scoring utilities for Tier 2 Cross-Sell & Upsell Orchestrator"""

from typing import Dict, List, Any


def calculate_business_value_score(opportunity: Dict[str, Any]) -> float:
    """
    Calculate business value score based on price and margin.

    Higher price and higher margin = higher score.

    Args:
        opportunity: Opportunity dict with price and margin

    Returns:
        Business value score (0.0 - 10.0)
    """
    price = opportunity.get("price", 0.0)
    margin = opportunity.get("margin", "medium")

    # Price component (0-5 points): Higher price = higher score
    # Tier 2 price range: $25-$60
    price_score = min(5.0, (price - 25.0) / 7.0)  # Normalize to 0-5
    price_score = max(0.0, price_score)  # Ensure non-negative

    # Margin component (0-5 points)
    margin_scores = {
        "low": 1.0,
        "medium": 3.0,
        "high": 5.0,
        "premium": 5.0
    }
    margin_score = margin_scores.get(margin.lower(), 3.0)

    return price_score + margin_score


def calculate_customer_fit_score(
    opportunity: Dict[str, Any],
    customer: Dict[str, Any]
) -> float:
    """
    Calculate customer fit score based on price sensitivity and tier preference.

    Args:
        opportunity: Opportunity dict with price
        customer: Customer dict with price_sensitivity and tier_preference

    Returns:
        Customer fit score (0.0 - 10.0)
    """
    price = opportunity.get("price", 0.0)
    price_sensitivity = customer.get("price_sensitivity", "medium")
    tier_preference = customer.get("tier_preference", 2)

    # Price sensitivity component (0-5 points)
    # Low sensitivity = higher score for expensive products
    # High sensitivity = higher score for cheaper products
    if price_sensitivity == "low":
        # Prefer higher prices
        price_fit = min(5.0, (price - 25.0) / 7.0)
    elif price_sensitivity == "high":
        # Prefer lower prices
        price_fit = min(5.0, (60.0 - price) / 7.0)
    else:  # medium
        # Balanced
        price_fit = 3.0

    price_fit = max(0.0, price_fit)

    # Tier preference component (0-5 points)
    # If customer prefers Tier 2, they're more likely to buy Tier 2 products
    if tier_preference >= 2:
        tier_fit = 5.0
    elif tier_preference == 1:
        tier_fit = 2.0  # Lower fit for Tier 1 customers
    else:
        tier_fit = 3.0

    return price_fit + tier_fit


def calculate_routine_completeness_score(
    opportunity: Dict[str, Any],
    customer: Dict[str, Any],
    routine_gaps: List[str],
    routine_step_gaps: List[int]
) -> float:
    """
    Calculate routine completeness score based on how important this product is for routine.

    Args:
        opportunity: Opportunity dict with category and routine_step
        customer: Customer dict
        routine_gaps: List of missing essential categories
        routine_step_gaps: List of missing routine steps

    Returns:
        Routine completeness score (0.0 - 10.0)
    """
    category = opportunity.get("category", "")
    routine_step = opportunity.get("routine_step", 0)
    recommendation_type = opportunity.get("recommendation_type", "")

    score = 0.0

    # Essential category gap (high priority)
    essential_categories = ["cleanser", "moisturizer", "spf"]
    if category in essential_categories and category in routine_gaps:
        score += 5.0  # High priority for essential categories

    # Routine step gap (medium priority)
    if routine_step in routine_step_gaps:
        score += 3.0

    # Recommendation type bonus
    if recommendation_type == "routine_step_gap":
        score += 2.0

    return min(10.0, score)


def calculate_replenishment_urgency_score(
    opportunity: Dict[str, Any]
) -> float:
    """
    Calculate replenishment urgency score based on days since purchase.

    Args:
        opportunity: Opportunity dict with replenishment_due, approaching_replenishment,
                    days_since_purchase, replenishment_cycle_days

    Returns:
        Replenishment urgency score (0.0 - 10.0)
    """
    if opportunity.get("recommendation_type") != "replenishment":
        return 0.0

    replenishment_due = opportunity.get("replenishment_due", False)
    approaching_replenishment = opportunity.get("approaching_replenishment", False)
    days_since_purchase = opportunity.get("days_since_purchase", 0)
    cycle_days = opportunity.get("replenishment_cycle_days", 30)

    if replenishment_due:
        # Overdue = highest urgency
        return 10.0
    elif approaching_replenishment:
        # Approaching = high urgency
        days_until_due = cycle_days - days_since_purchase
        # Closer to due date = higher score
        return 7.0 + (7.0 - days_until_due) / 7.0 * 3.0
    else:
        return 0.0


def calculate_ingredient_match_score(
    opportunity: Dict[str, Any],
    customer: Dict[str, Any]
) -> float:
    """
    Calculate ingredient match score based on concern matching.

    Args:
        opportunity: Opportunity dict with target_concerns and match_score
        customer: Customer dict with skin_concerns

    Returns:
        Ingredient match score (0.0 - 10.0)
    """
    # Use match_score if available (from ingredient-based opportunities)
    match_score = opportunity.get("match_score", 0.0)

    if match_score > 0:
        # Convert 0.0-1.0 match_score to 0.0-10.0
        return match_score * 10.0

    # If no match_score, check manually
    customer_concerns = set(customer.get("skin_concerns", []))
    product_concerns = set(opportunity.get("target_concerns", []))

    if customer_concerns & product_concerns:
        # Exact match
        return 10.0
    elif product_concerns:
        # Partial match (product has concerns but not exact match)
        return 5.0
    else:
        return 0.0


def calculate_upgrade_readiness_score(
    opportunity: Dict[str, Any],
    customer: Dict[str, Any]
) -> float:
    """
    Calculate upgrade readiness score based on customer's tier preference and upgrade path.

    Args:
        opportunity: Opportunity dict with recommendation_type and upgrades_from
        customer: Customer dict with tier_preference

    Returns:
        Upgrade readiness score (0.0 - 10.0)
    """
    if opportunity.get("recommendation_type") != "upgrade":
        return 0.0

    tier_preference = customer.get("tier_preference", 1)
    upgrades_from = opportunity.get("upgrades_from", [])

    # Only score if customer has tier_preference >= 2
    if tier_preference < 2:
        return 0.0

    # If customer prefers Tier 2 and product has upgrade path, high readiness
    if tier_preference >= 2 and upgrades_from:
        return 10.0

    return 0.0


def score_opportunity(
    opportunity: Dict[str, Any],
    customer: Dict[str, Any],
    routine_gaps: List[str],
    routine_step_gaps: List[int],
    product: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Score an opportunity multi-dimensionally.

    Calculates all individual scores and a final weighted score.

    Args:
        opportunity: Opportunity dict
        customer: Customer dict
        routine_gaps: List of missing essential categories
        routine_step_gaps: List of missing routine steps
        product: Product dict (for additional product details if needed)

    Returns:
        Opportunity dict with all scores added:
        - business_value_score
        - customer_fit_score
        - routine_completeness_score
        - replenishment_urgency_score
        - ingredient_match_score
        - upgrade_readiness_score
        - final_score (weighted combination)
    """
    # Calculate individual scores
    business_value_score = calculate_business_value_score(opportunity)
    customer_fit_score = calculate_customer_fit_score(opportunity, customer)
    routine_completeness_score = calculate_routine_completeness_score(
        opportunity, customer, routine_gaps, routine_step_gaps
    )
    replenishment_urgency_score = calculate_replenishment_urgency_score(opportunity)
    ingredient_match_score = calculate_ingredient_match_score(opportunity, customer)
    upgrade_readiness_score = calculate_upgrade_readiness_score(opportunity, customer)

    # Calculate final weighted score
    # Weights can be adjusted based on business priorities
    weights = {
        "business_value": 0.20,      # 20% - Revenue/margin
        "customer_fit": 0.15,        # 15% - Price sensitivity match
        "routine_completeness": 0.20, # 20% - Routine gap importance
        "replenishment_urgency": 0.25, # 25% - Highest priority (customer needs to repurchase)
        "ingredient_match": 0.15,   # 15% - Concern matching
        "upgrade_readiness": 0.05    # 5% - Upgrade path
    }

    final_score = (
        business_value_score * weights["business_value"] +
        customer_fit_score * weights["customer_fit"] +
        routine_completeness_score * weights["routine_completeness"] +
        replenishment_urgency_score * weights["replenishment_urgency"] +
        ingredient_match_score * weights["ingredient_match"] +
        upgrade_readiness_score * weights["upgrade_readiness"]
    )

    # Add scores to opportunity
    scored_opportunity = opportunity.copy()
    scored_opportunity.update({
        "business_value_score": round(business_value_score, 2),
        "customer_fit_score": round(customer_fit_score, 2),
        "routine_completeness_score": round(routine_completeness_score, 2),
        "replenishment_urgency_score": round(replenishment_urgency_score, 2),
        "ingredient_match_score": round(ingredient_match_score, 2),
        "upgrade_readiness_score": round(upgrade_readiness_score, 2),
        "final_score": round(final_score, 2)
    })

    return scored_opportunity



# Tests for scoring utilities

In [ ]:
"""Tests for scoring utilities"""

from agents.tier2_crosssell_upsell.utilities.scoring import (
    calculate_business_value_score,
    calculate_customer_fit_score,
    calculate_routine_completeness_score,
    calculate_replenishment_urgency_score,
    calculate_ingredient_match_score,
    calculate_upgrade_readiness_score,
    score_opportunity
)
from agents.tier2_crosssell_upsell.utilities.data_loading import (
    load_customer_data,
    load_product_catalog,
    build_product_lookup
)
from agents.tier2_crosssell_upsell.utilities.opportunity_detection import (
    find_cross_sell_opportunities,
    find_ingredient_based_opportunities
)
from agents.tier2_crosssell_upsell.utilities.routine_analysis import (
    analyze_customer_routine
)


def test_calculate_business_value_score():
    """Test calculate_business_value_score calculates correctly"""
    print("\n=== Testing calculate_business_value_score ===")

    # Test high-value product
    high_value_opp = {
        "price": 50.0,
        "margin": "high"
    }
    score = calculate_business_value_score(high_value_opp)
    assert 0.0 <= score <= 10.0, "Score should be between 0 and 10"
    assert score > 5.0, "High value product should score > 5"

    # Test medium-value product
    medium_value_opp = {
        "price": 35.0,
        "margin": "medium"
    }
    score_medium = calculate_business_value_score(medium_value_opp)
    assert 0.0 <= score_medium <= 10.0, "Score should be between 0 and 10"

    # Test low-value product
    low_value_opp = {
        "price": 25.0,
        "margin": "low"
    }
    score_low = calculate_business_value_score(low_value_opp)
    assert 0.0 <= score_low <= 10.0, "Score should be between 0 and 10"
    assert score_low < score_medium, "Low value should score lower than medium"

    print("✅ calculate_business_value_score test passed!")
    print(f"   High value: {score}, Medium: {score_medium}, Low: {score_low}")

    return score


def test_calculate_customer_fit_score():
    """Test calculate_customer_fit_score matches price sensitivity"""
    print("\n=== Testing calculate_customer_fit_score ===")

    # Test low price sensitivity customer (prefers higher prices)
    customer_low_sensitivity = {
        "price_sensitivity": "low",
        "tier_preference": 2
    }
    expensive_opp = {"price": 55.0}
    score_expensive = calculate_customer_fit_score(expensive_opp, customer_low_sensitivity)

    # Test high price sensitivity customer (prefers lower prices)
    customer_high_sensitivity = {
        "price_sensitivity": "high",
        "tier_preference": 2
    }
    cheap_opp = {"price": 28.0}
    score_cheap = calculate_customer_fit_score(cheap_opp, customer_high_sensitivity)

    assert 0.0 <= score_expensive <= 10.0, "Score should be between 0 and 10"
    assert 0.0 <= score_cheap <= 10.0, "Score should be between 0 and 10"

    print("✅ calculate_customer_fit_score test passed!")
    print(f"   Low sensitivity + expensive: {score_expensive}")
    print(f"   High sensitivity + cheap: {score_cheap}")

    return score_expensive


def test_calculate_routine_completeness_score():
    """Test calculate_routine_completeness_score prioritizes essential categories"""
    print("\n=== Testing calculate_routine_completeness_score ===")

    # Test essential category gap
    essential_opp = {
        "category": "cleanser",
        "routine_step": 1,
        "recommendation_type": "routine_step_gap"
    }
    customer = {}
    routine_gaps = ["cleanser"]  # Missing cleanser
    routine_step_gaps = [1]

    score = calculate_routine_completeness_score(
        essential_opp, customer, routine_gaps, routine_step_gaps
    )
    assert 0.0 <= score <= 10.0, "Score should be between 0 and 10"
    assert score > 5.0, "Essential category gap should score high"

    # Test non-essential category
    non_essential_opp = {
        "category": "mask",
        "routine_step": 3,
        "recommendation_type": "routine_step_gap"
    }
    routine_gaps_empty = []
    routine_step_gaps_3 = [3]

    score_non_essential = calculate_routine_completeness_score(
        non_essential_opp, customer, routine_gaps_empty, routine_step_gaps_3
    )
    assert 0.0 <= score_non_essential <= 10.0, "Score should be between 0 and 10"

    print("✅ calculate_routine_completeness_score test passed!")
    print(f"   Essential category: {score}, Non-essential: {score_non_essential}")

    return score


def test_calculate_replenishment_urgency_score():
    """Test calculate_replenishment_urgency_score prioritizes overdue products"""
    print("\n=== Testing calculate_replenishment_urgency_score ===")

    # Test overdue replenishment
    overdue_opp = {
        "recommendation_type": "replenishment",
        "replenishment_due": True,
        "days_since_purchase": 35,
        "replenishment_cycle_days": 30
    }
    score_overdue = calculate_replenishment_urgency_score(overdue_opp)
    assert score_overdue == 10.0, "Overdue should score 10.0"

    # Test approaching replenishment
    approaching_opp = {
        "recommendation_type": "replenishment",
        "replenishment_due": False,
        "approaching_replenishment": True,
        "days_since_purchase": 25,
        "replenishment_cycle_days": 30
    }
    score_approaching = calculate_replenishment_urgency_score(approaching_opp)
    assert 0.0 <= score_approaching <= 10.0, "Score should be between 0 and 10"
    assert score_approaching > 5.0, "Approaching should score > 5"

    # Test non-replenishment opportunity
    non_replenishment_opp = {
        "recommendation_type": "ingredient_match"
    }
    score_non = calculate_replenishment_urgency_score(non_replenishment_opp)
    assert score_non == 0.0, "Non-replenishment should score 0"

    print("✅ calculate_replenishment_urgency_score test passed!")
    print(f"   Overdue: {score_overdue}, Approaching: {score_approaching}, Non: {score_non}")

    return score_overdue


def test_calculate_ingredient_match_score():
    """Test calculate_ingredient_match_score matches concerns"""
    print("\n=== Testing calculate_ingredient_match_score ===")

    # Test with match_score (from ingredient-based opportunities)
    customer = {"skin_concerns": ["dryness", "sensitivity"]}
    matched_opp = {
        "target_concerns": ["dryness", "sensitivity"],
        "match_score": 1.0,
        "matched_concerns": ["dryness", "sensitivity"]
    }
    score = calculate_ingredient_match_score(matched_opp, customer)
    assert score == 10.0, "Exact match should score 10.0"

    # Test without match_score (manual check)
    manual_opp = {
        "target_concerns": ["dryness", "sensitivity"]
    }
    score_manual = calculate_ingredient_match_score(manual_opp, customer)
    assert score_manual == 10.0, "Manual exact match should score 10.0"

    # Test no match
    no_match_opp = {
        "target_concerns": ["acne", "oiliness"]
    }
    score_no_match = calculate_ingredient_match_score(no_match_opp, customer)
    assert score_no_match == 0.0, "No match should score 0.0"

    print("✅ calculate_ingredient_match_score test passed!")
    print(f"   Matched: {score}, Manual: {score_manual}, No match: {score_no_match}")

    return score


def test_calculate_upgrade_readiness_score():
    """Test calculate_upgrade_readiness_score checks tier preference"""
    print("\n=== Testing calculate_upgrade_readiness_score ===")

    # Test upgrade opportunity with tier_preference >= 2
    customer_tier2 = {"tier_preference": 2}
    upgrade_opp = {
        "recommendation_type": "upgrade",
        "upgrades_from": ["P001"]
    }
    score = calculate_upgrade_readiness_score(upgrade_opp, customer_tier2)
    assert score == 10.0, "Tier 2 customer with upgrade path should score 10.0"

    # Test upgrade opportunity with tier_preference < 2
    customer_tier1 = {"tier_preference": 1}
    score_tier1 = calculate_upgrade_readiness_score(upgrade_opp, customer_tier1)
    assert score_tier1 == 0.0, "Tier 1 customer should score 0.0"

    # Test non-upgrade opportunity
    non_upgrade_opp = {
        "recommendation_type": "ingredient_match"
    }
    score_non = calculate_upgrade_readiness_score(non_upgrade_opp, customer_tier2)
    assert score_non == 0.0, "Non-upgrade should score 0.0"

    print("✅ calculate_upgrade_readiness_score test passed!")
    print(f"   Tier 2 customer: {score}, Tier 1 customer: {score_tier1}, Non-upgrade: {score_non}")

    return score


def test_score_opportunity():
    """Test score_opportunity calculates all scores"""
    print("\n=== Testing score_opportunity ===")

    # Load real data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Get a real opportunity
    routine_analysis = analyze_customer_routine(customer, product_lookup)
    opportunities = find_cross_sell_opportunities(
        routine_analysis["customer_routine_steps"],
        routine_analysis["customer_products"],
        products,
        product_lookup
    )

    if not opportunities:
        print("   No opportunities found, skipping test")
        return

    opportunity = opportunities[0]
    product = product_lookup[opportunity["product_id"]]

    # Score the opportunity
    scored = score_opportunity(
        opportunity,
        customer,
        [],  # routine_gaps
        [2],  # routine_step_gaps (C001 missing step 2)
        product
    )

    # Verify all scores are present
    assert "business_value_score" in scored, "Should have business_value_score"
    assert "customer_fit_score" in scored, "Should have customer_fit_score"
    assert "routine_completeness_score" in scored, "Should have routine_completeness_score"
    assert "replenishment_urgency_score" in scored, "Should have replenishment_urgency_score"
    assert "ingredient_match_score" in scored, "Should have ingredient_match_score"
    assert "upgrade_readiness_score" in scored, "Should have upgrade_readiness_score"
    assert "final_score" in scored, "Should have final_score"

    # Verify scores are in valid range
    for score_name in ["business_value_score", "customer_fit_score", "routine_completeness_score",
                       "replenishment_urgency_score", "ingredient_match_score", "upgrade_readiness_score"]:
        score_value = scored[score_name]
        assert 0.0 <= score_value <= 10.0, f"{score_name} should be between 0 and 10"

    assert 0.0 <= scored["final_score"] <= 10.0, "final_score should be between 0 and 10"

    print("✅ score_opportunity test passed!")
    print(f"   Product: {scored['product_name']}")
    print(f"   Business value: {scored['business_value_score']}")
    print(f"   Customer fit: {scored['customer_fit_score']}")
    print(f"   Routine completeness: {scored['routine_completeness_score']}")
    print(f"   Final score: {scored['final_score']}")

    return scored


def test_scoring_integration():
    """Test scoring with real opportunities"""
    print("\n=== Testing scoring integration ===")

    # Load real data
    customer = load_customer_data("C001")
    products = load_product_catalog()
    product_lookup = build_product_lookup(products)

    # Get opportunities
    routine_analysis = analyze_customer_routine(customer, product_lookup)
    cross_sell = find_cross_sell_opportunities(
        routine_analysis["customer_routine_steps"],
        routine_analysis["customer_products"],
        products,
        product_lookup
    )
    ingredient = find_ingredient_based_opportunities(
        customer,
        routine_analysis["customer_products"],
        products,
        product_lookup
    )

    # Score all opportunities
    scored_opportunities = []
    routine_gaps = []  # C001 has all essential categories
    routine_step_gaps = [2]  # C001 missing step 2

    for opp in cross_sell + ingredient:
        product = product_lookup[opp["product_id"]]
        scored = score_opportunity(
            opp, customer, routine_gaps, routine_step_gaps, product
        )
        scored_opportunities.append(scored)

    # Verify all scored
    assert len(scored_opportunities) > 0, "Should have scored opportunities"

    # Sort by final_score
    scored_opportunities.sort(key=lambda x: x["final_score"], reverse=True)

    print("✅ Scoring integration test passed!")
    print(f"   Opportunities scored: {len(scored_opportunities)}")
    print(f"   Top opportunity: {scored_opportunities[0]['product_name']} "
          f"(score: {scored_opportunities[0]['final_score']})")

    return scored_opportunities


if __name__ == "__main__":
    """Run all tests"""
    print("=" * 60)
    print("Testing Tier 2 Scoring Utilities")
    print("=" * 60)

    # Test individual score functions
    test_calculate_business_value_score()
    test_calculate_customer_fit_score()
    test_calculate_routine_completeness_score()
    test_calculate_replenishment_urgency_score()
    test_calculate_ingredient_match_score()
    test_calculate_upgrade_readiness_score()

    # Test main scoring function
    test_score_opportunity()

    # Integration test
    test_scoring_integration()

    print("\n" + "=" * 60)
    print("✅ All scoring utility tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.tests.test_scoring
============================================================
Testing Tier 2 Scoring Utilities
============================================================

=== Testing calculate_business_value_score ===
✅ calculate_business_value_score test passed!
   High value: 8.571428571428571, Medium: 4.428571428571429, Low: 1.0

=== Testing calculate_customer_fit_score ===
✅ calculate_customer_fit_score test passed!
   Low sensitivity + expensive: 9.285714285714285
   High sensitivity + cheap: 9.571428571428571

=== Testing calculate_routine_completeness_score ===
✅ calculate_routine_completeness_score test passed!
   Essential category: 10.0, Non-essential: 5.0

=== Testing calculate_replenishment_urgency_score ===
✅ calculate_replenishment_urgency_score test passed!
   Overdue: 10.0, Approaching: 7.857142857142857, Non: 0.0

=== Testing calculate_ingredient_match_score ===
✅ calculate_ingredient_match_score test passed!
   Matched: 10.0, Manual: 10.0, No match: 0.0

=== Testing calculate_upgrade_readiness_score ===
✅ calculate_upgrade_readiness_score test passed!
   Tier 2 customer: 10.0, Tier 1 customer: 0.0, Non-upgrade: 0.0

=== Testing score_opportunity ===
✅ score_opportunity test passed!
   Product: HydraBalance Prep Toner
   Business value: 4.0
   Customer fit: 8.0
   Routine completeness: 5.0
   Final score: 4.5

=== Testing scoring integration ===
✅ Scoring integration test passed!
   Opportunities scored: 4
   Top opportunity: HydraBalance Prep Toner (score: 4.5)

============================================================
✅ All scoring utility tests passed!
============================================================


# Scoring Node

In [ ]:
def scoring_node(state: Tier2CrossSellUpsellState) -> Dict[str, Any]:
    """
    Scoring Node: Score all opportunities multi-dimensionally.

    This node orchestrates scoring using the scoring utilities.
    It scores all opportunities from all types (cross-sell, ingredient, upgrade, replenishment).

    Args:
        state: Current state with all opportunities, customer_data, product_lookup,
               routine_gaps, routine_step_gaps

    Returns:
        Updated state with scored_opportunities (all opportunities with scores)
    """
    errors = state.get("errors", [])

    # Get required data from state
    customer_data = state.get("customer_data")
    product_lookup = state.get("product_lookup")
    routine_gaps = state.get("routine_gaps", [])
    routine_step_gaps = state.get("routine_step_gaps", [])

    # Get all opportunities
    cross_sell = state.get("cross_sell_opportunities", [])
    ingredient = state.get("ingredient_opportunities", [])
    upgrade = state.get("upgrade_opportunities", [])
    replenishment = state.get("replenishment_opportunities", [])
    bundle = state.get("bundle_opportunities", [])

    if not customer_data:
        return {
            "errors": errors + ["scoring_node: customer_data is required"]
        }

    if not product_lookup:
        return {
            "errors": errors + ["scoring_node: product_lookup is required"]
        }

    try:
        # Combine all opportunities
        all_opportunities = cross_sell + ingredient + upgrade + replenishment + bundle

        # Score each opportunity
        scored_opportunities = []
        for opportunity in all_opportunities:
            product_id = opportunity.get("product_id")
            product = product_lookup.get(product_id)

            if not product:
                continue  # Skip if product not found

            # Score the opportunity
            scored = score_opportunity(
                opportunity,
                customer_data,
                routine_gaps,
                routine_step_gaps,
                product
            )
            scored_opportunities.append(scored)

    except Exception as e:
        return {
            "errors": errors + [f"scoring_node: Error during scoring: {str(e)}"]
        }

    return {
        "scored_opportunities": scored_opportunities,
        "errors": errors
    }


def ranking_node(state: Tier2CrossSellUpsellState) -> Dict[str, Any]:
    """
    Ranking Node: Rank opportunities by final score and select top N.

    This node sorts all scored opportunities by final_score and selects the top N
    based on configuration. It also generates opportunity summary metrics.

    Args:
        state: Current state with scored_opportunities

    Returns:
        Updated state with:
        - ranked_opportunities: Sorted by final_score (descending)
        - top_opportunities: Top N opportunities
        - opportunity_summary: Summary metrics
    """
    errors = state.get("errors", [])

    # Get required data
    scored_opportunities = state.get("scored_opportunities", [])

    if not scored_opportunities:
        return {
            "ranked_opportunities": [],
            "top_opportunities": [],
            "opportunity_summary": {
                "total_opportunities": 0,
                "total_potential_revenue": 0.0
            },
            "errors": errors
        }

    # Get config
    config = Tier2CrossSellUpsellConfig()
    top_n = config.top_n_opportunities
    high_value_threshold = config.high_value_score_threshold

    try:
        # Sort by final_score (descending)
        ranked_opportunities = sorted(
            scored_opportunities,
            key=lambda x: x.get("final_score", 0.0),
            reverse=True
        )

        # Select top N
        top_opportunities = ranked_opportunities[:top_n]

        # Calculate summary metrics
        # Count by type
        cross_sell_count = len(state.get("cross_sell_opportunities", []))
        ingredient_count = len(state.get("ingredient_opportunities", []))
        upgrade_count = len(state.get("upgrade_opportunities", []))
        replenishment_count = len(state.get("replenishment_opportunities", []))

        # Calculate total potential revenue
        total_potential_revenue = sum(
            opp.get("price", 0.0) for opp in scored_opportunities
        )

        # Count high-value opportunities
        high_value_count = sum(
            1 for opp in scored_opportunities
            if opp.get("final_score", 0.0) > high_value_threshold
        )

        # Routine completeness metrics
        routine_steps_completed = len(state.get("customer_routine_steps", []))
        routine_steps_total = config.routine_steps_total
        routine_completeness_percent = (
            (routine_steps_completed / routine_steps_total * 100)
            if routine_steps_total > 0 else 0.0
        )

        # Replenishment urgency count
        replenishment_urgency_count = sum(
            1 for need in state.get("replenishment_needs", [])
            if need.get("replenishment_due", False) or need.get("approaching_replenishment", False)
        )

        opportunity_summary = {
            "total_cross_sell_opportunities": cross_sell_count,
            "total_ingredient_opportunities": ingredient_count,
            "total_upgrade_opportunities": upgrade_count,
            "total_replenishment_opportunities": replenishment_count,
            "total_opportunities": len(scored_opportunities),
            "total_potential_revenue": round(total_potential_revenue, 2),
            "routine_completeness_percent": round(routine_completeness_percent, 1),
            "routine_steps_completed": routine_steps_completed,
            "routine_steps_total": routine_steps_total,
            "replenishment_urgency_count": replenishment_urgency_count,
            "high_value_opportunities": high_value_count
        }

    except Exception as e:
        return {
            "errors": errors + [f"ranking_node: Error during ranking: {str(e)}"]
        }

    return {
        "ranked_opportunities": ranked_opportunities,
        "top_opportunities": top_opportunities,
        "opportunity_summary": opportunity_summary,
        "errors": errors
    }



# Test Scoring Ranking Node

In [ ]:
def test_scoring_node():
    """Test scoring_node scores all opportunities"""
    print("\n=== Testing scoring_node ===")

    # Set up state with all previous nodes
    state: Tier2CrossSellUpsellState = {
        "customer_id": "C001",
        "goal": {"customer_id": "C001"},
        "errors": []
    }

    # Load data, analyze routine, detect opportunities
    state = {**state, **data_loading_node(state)}
    state = {**state, **routine_analysis_node(state)}
    state = {**state, **opportunity_detection_node(state)}

    assert "cross_sell_opportunities" in state, "State should have opportunities"

    # Test scoring_node
    result = scoring_node(state)

    # Verify scored_opportunities
    assert "scored_opportunities" in result, "Should return scored_opportunities"
    assert isinstance(result["scored_opportunities"], list), "scored_opportunities should be a list"

    # Verify each scored opportunity has all score fields
    for scored in result["scored_opportunities"]:
        assert "final_score" in scored, "Each scored opportunity should have final_score"
        assert "business_value_score" in scored, "Each should have business_value_score"
        assert "customer_fit_score" in scored, "Each should have customer_fit_score"
        assert "routine_completeness_score" in scored, "Each should have routine_completeness_score"
        assert "replenishment_urgency_score" in scored, "Each should have replenishment_urgency_score"
        assert "ingredient_match_score" in scored, "Each should have ingredient_match_score"
        assert "upgrade_readiness_score" in scored, "Each should have upgrade_readiness_score"

        # Verify scores are valid
        assert 0.0 <= scored["final_score"] <= 10.0, "final_score should be 0-10"

    print("✅ scoring_node test passed!")
    print(f"   Opportunities scored: {len(result['scored_opportunities'])}")
    if result["scored_opportunities"]:
        top = result["scored_opportunities"][0]
        print(f"   Top scored: {top.get('product_name', 'N/A')} (score: {top.get('final_score', 0)})")

    return result


def test_ranking_node():
    """Test ranking_node ranks and selects top opportunities"""
    print("\n=== Testing ranking_node ===")

    # Set up state with scored opportunities
    state: Tier2CrossSellUpsellState = {
        "customer_id": "C001",
        "goal": {"customer_id": "C001"},
        "errors": []
    }

    # Load data, analyze, detect, score
    state = {**state, **data_loading_node(state)}
    state = {**state, **routine_analysis_node(state)}
    state = {**state, **opportunity_detection_node(state)}
    state = {**state, **scoring_node(state)}

    assert "scored_opportunities" in state, "State should have scored_opportunities"

    # Test ranking_node
    result = ranking_node(state)

    # Verify ranked_opportunities
    assert "ranked_opportunities" in result, "Should return ranked_opportunities"
    assert "top_opportunities" in result, "Should return top_opportunities"
    assert "opportunity_summary" in result, "Should return opportunity_summary"

    # Verify ranking is correct (descending by final_score)
    ranked = result["ranked_opportunities"]
    if len(ranked) > 1:
        for i in range(len(ranked) - 1):
            assert ranked[i]["final_score"] >= ranked[i + 1]["final_score"], \
                "Ranked opportunities should be in descending order"

    # Verify top_opportunities is subset of ranked
    top = result["top_opportunities"]
    assert len(top) <= len(ranked), "top_opportunities should be <= ranked_opportunities"
    assert len(top) <= 3, "top_opportunities should be <= 3 (config default)"

    # Verify summary metrics
    summary = result["opportunity_summary"]
    assert "total_opportunities" in summary, "Summary should have total_opportunities"
    assert "total_potential_revenue" in summary, "Summary should have total_potential_revenue"
    assert "routine_completeness_percent" in summary, "Summary should have routine_completeness_percent"

    print("✅ ranking_node test passed!")
    print(f"   Ranked opportunities: {len(ranked)}")
    print(f"   Top opportunities: {len(top)}")
    print(f"   Total potential revenue: ${summary['total_potential_revenue']}")
    print(f"   High-value opportunities: {summary.get('high_value_opportunities', 0)}")

    return result


def test_full_integration_through_ranking():
    """Test full integration: goal → ... → scoring → ranking"""
    print("\n=== Testing full integration through ranking ===")

    # Start with just customer_id
    state: Tier2CrossSellUpsellState = {
        "customer_id": "C001",
        "errors": []
    }

    # Run all nodes in sequence
    state = {**state, **goal_node(state)}
    state = {**state, **planning_node(state)}
    state = {**state, **data_loading_node(state)}
    state = {**state, **routine_analysis_node(state)}
    state = {**state, **opportunity_detection_node(state)}
    state = {**state, **scoring_node(state)}
    state = {**state, **ranking_node(state)}

    # Verify final state
    assert "ranked_opportunities" in state, "State should have ranked_opportunities"
    assert "top_opportunities" in state, "State should have top_opportunities"
    assert "opportunity_summary" in state, "State should have opportunity_summary"

    print("✅ Full integration test passed!")
    print(f"   Customer: {state['customer_data']['name']} ({state['customer_id']})")
    print(f"   Total opportunities: {state['opportunity_summary']['total_opportunities']}")
    print(f"   Top opportunities: {len(state['top_opportunities'])}")
    print(f"   Total potential revenue: ${state['opportunity_summary']['total_potential_revenue']}")

    if state["top_opportunities"]:
        print(f"   #1 Recommendation: {state['top_opportunities'][0]['product_name']} "
              f"(score: {state['top_opportunities'][0]['final_score']})")

    return state


# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_030_CrossSell_Upsell_Orchestrator % python3 -m agents.tier2_crosssell_upsell.tests.test_nodes
============================================================
Testing Tier 2 Cross-Sell & Upsell Orchestrator Nodes
============================================================

=== Testing goal_node ===
✅ goal_node test passed!
   Goal: Identify cross-sell and upsell opportunities for Tier 2 customer
   Focus areas: ['routine_completeness', 'ingredient_matching', 'upgrade_opportunities', 'replenishment_needs', 'routine_step_adjacency']

=== Testing goal_node with missing customer_id ===
✅ goal_node error handling test passed!
   Error: goal_node: customer_id is required

=== Testing planning_node ===
✅ planning_node test passed!
   Plan has 6 steps:
      Step 1: data_loading
      Step 2: routine_analysis
      Step 3: opportunity_detection
      Step 4: scoring
      Step 5: ranking
      Step 6: report_generation

=== Testing planning_node with missing goal ===
✅ planning_node error handling test passed!
   Error: planning_node: goal is required

=== Testing goal_node → planning_node integration ===
✅ Integration test passed!
   Customer ID: C001
   Goal: Identify cross-sell and upsell opportunities for Tier 2 customer
   Plan steps: 6

=== Testing data_loading_node ===
✅ data_loading_node test passed!
   Customer: Sarah Lee (C001)
   Products in catalog: 20
   Lookup size: 20

=== Testing data_loading_node with missing customer_id ===
✅ data_loading_node error handling test passed!
   Error: data_loading_node: customer_id is required

=== Testing data_loading_node with customer not found ===
✅ data_loading_node (customer not found) test passed!
   Error: data_loading_node: Customer C999 not found

=== Testing goal_node → planning_node → data_loading_node integration ===
✅ Integration test passed!
   Customer ID: C001
   Customer: Sarah Lee
   Products in catalog: 20
   Plan steps: 6

=== Testing routine_analysis_node ===
✅ routine_analysis_node test passed!
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]
   Missing categories: []
   Missing steps: [2]
   Replenishment needs: 6

=== Testing routine_analysis_node with missing data ===
✅ routine_analysis_node error handling test passed!
   Error: routine_analysis_node: customer_data is required

=== Testing full integration (goal → planning → data_loading → routine_analysis) ===
✅ Full integration test passed!
   Customer: Sarah Lee (C001)
   Products: 6
   Categories: ['cleanser', 'mask', 'moisturizer', 'serum', 'spf', 'toner']
   Routine steps: [1, 3, 4, 5]
   Missing categories: []
   Missing steps: [2]
   Replenishment needs: 6

=== Testing opportunity_detection_node ===
✅ opportunity_detection_node test passed!
   Cross-sell opportunities: 2
   Ingredient opportunities: 2
   Upgrade opportunities: 0
   Replenishment opportunities: 2
   Bundle opportunities: 0

=== Testing opportunity_detection_node with missing data ===
✅ opportunity_detection_node error handling test passed!
   Error: opportunity_detection_node: customer_data is required

=== Testing full integration through opportunity_detection ===
✅ Full integration test passed!
   Customer: Sarah Lee (C001)
   Cross-sell: 2
   Ingredient: 2
   Upgrade: 0
   Replenishment: 2
   Total opportunities: 6

=== Testing scoring_node ===
✅ scoring_node test passed!
   Opportunities scored: 6
   Top scored: HydraBalance Prep Toner (score: 4.5)

=== Testing ranking_node ===
✅ ranking_node test passed!
   Ranked opportunities: 6
   Top opportunities: 3
   Total potential revenue: $198.0
   High-value opportunities: 0

=== Testing full integration through ranking ===
✅ Full integration test passed!
   Customer: Sarah Lee (C001)
   Total opportunities: 6
   Top opportunities: 3
   Total potential revenue: $198.0
   #1 Recommendation: Barrier Repair Overnight Mask (score: 6.03)

============================================================
✅ All tests passed!
============================================================
